In [2]:
import duckdb
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import warnings
import json
warnings.filterwarnings('ignore')

In [ ]:

# Configuración
PERIODO_CORTE = 201912  # Entrenar hasta este periodo
TARGET_LAG = 2  # Predecir periodo +2

def load_and_prepare_data():
    """
    Carga y prepara los datos para el modelo
    """
    # Conectar a la base de datos DuckDB
    con = duckdb.connect(database='../input/db/labo3.duckdb')
    
    # Cargar datos de la tabla ventas_features_final
    query = """SELECT 
        antiguedad_cliente,
        antiguedad_producto,
        cust_request_qty,
        cust_request_tn,
        customer_id,
        periodo,
        plan_precios_cuidados,
        product_id,
        registro_sintetico,
        tn,
        coef_variacion_6m,
        delta_tn_12m,
        delta_tn_1m,
        delta_tn_3m,
        delta_tn_6m,
        is_max_tn_12m,
        is_max_tn_3m,
        is_max_tn_6m,
        is_min_tn_12m,
        is_min_tn_3m,
        is_min_tn_6m,
        ma_tn_12m,
        ma_tn_3m,
        ma_tn_6m,
        tendencia_6m,
        tn_lag_12m,
        tn_lag_1m,
        tn_lag_3m,
        tn_lag_6m,
        tn_mismo_mes_anio_anterior,
        trimestre,
        desviacion_vs_promedio_historico_mes,
        meses_desde_ultima_compra,
        promedio_historico_mes,
        stddev_historico_mes,
        cat1,
        cat2,
        -- cat3,
        brand
        -- sku_size 
    FROM ventas_features_final"""
    df = con.execute(query).df()
    
    # Cerrar conexión
    con.close()
    print(f"Datos originales: {df.shape}")
    print(f"Periodos únicos: {sorted(df['periodo'].unique())}")
    
    return df

def create_target_variable_correct(df):
    # Ordenar por cliente, producto, periodo
    df = df.sort_values(['customer_id', 'product_id', 'periodo'])
    
    # Target = tn individual en periodo +2
    df['target_tn'] = df.groupby(['customer_id', 'product_id'])['tn'].shift(-TARGET_LAG)
    
    return df

def create_target_variable_v_old(df):
    """
    Crea la variable objetivo (tn en periodo +2) agrupada por product_id
    """
    print("Creando variable objetivo...")
    
    # Agregar tn por product_id y periodo
    df_agg = df.groupby(['product_id', 'periodo']).agg({
        'tn': 'sum'
    }).reset_index()
    
    # Ordenar por product_id y periodo
    df_agg = df_agg.sort_values(['product_id', 'periodo'])
    
    # Crear target variable (tn en periodo +2)
    df_agg['target_tn'] = df_agg.groupby('product_id')['tn'].shift(-TARGET_LAG)
    
    # Merge de vuelta con el dataset original
    df = df.merge(df_agg[['product_id', 'periodo', 'target_tn']], 
                  on=['product_id', 'periodo'], how='left')
    
    print(f"Registros con target válido: {df['target_tn'].notna().sum()}")
    
    return df

def prepare_features(df):
    """
    Prepara las features excluyendo variables key_*, fechas y separando categóricas
    """
    print("Preparando features...")
    
    # Variables a excluir
    exclude_vars = [col for col in df.columns if col.startswith('key_')]
    exclude_vars.extend(['target_tn', 'tn'])  # Excluir también el target y la variable original
    
    # Excluir variables de fecha que no son útiles como features numéricas
    date_vars = ['PVC', 'PVP', 'UVC', 'UVP', 'periodo_fecha']
    exclude_vars.extend([col for col in date_vars if col in df.columns])
    
    # Variables categóricas
    categorical_vars = ['cat1', 'cat2', 'cat3', 'brand']
    
    # Variables numéricas (todas las demás excepto las excluidas)
    all_vars = set(df.columns)
    numeric_vars = list(all_vars - set(exclude_vars) - set(categorical_vars))
    
    print(f"Variables categóricas ({len(categorical_vars)}): {categorical_vars}")
    print(f"Variables numéricas ({len(numeric_vars)}): {numeric_vars[:10]}...")  # Mostrar solo las primeras 10
    print(f"Variables excluidas ({len(exclude_vars)}): {exclude_vars}")
    
    return numeric_vars, categorical_vars, exclude_vars

def prepare_features_fixed(df):
    """
    Función CORREGIDA que FUERZA la exclusión de variables problemáticas
    """
    print("Preparando features (versión CORREGIDA)...")
    
    # 1. EXCLUSIÓN FORZADA - ESTAS VARIABLES NO PUEDEN SER FEATURES
    forced_excludes = [
        'product_id',           # ID del producto - NO es feature
        'customer_id',          # ID del cliente - NO es feature  
        'periodo',              # Periodo - NO es feature
        'target_tn',            # Variable objetivo
        'tn',                   # Variable original
        'registro_sintetico'    # Variable sintética
    ]
    
    # 2. Variables categóricas
    categorical_vars = ['cat1', 'cat2', 'brand']  # Solo las que existen
    
    # 3. Calcular variables numéricas = TODO lo demás
    all_columns = set(df.columns)
    excluded_set = set(forced_excludes)
    categorical_set = set(categorical_vars)
    
    numeric_vars = list(all_columns - excluded_set - categorical_set)
    
    # 4. VERIFICACIÓN CRÍTICA
    contaminated = []
    for var in numeric_vars:
        if var in ['product_id', 'customer_id', 'periodo']:
            contaminated.append(var)
    
    if contaminated:
        print(f"🚨 ERROR CRÍTICO: Variables prohibidas detectadas: {contaminated}")
        print("🛑 DETENIENDO EJECUCIÓN")
        return None, None, None
    
    print(f"✅ VERIFICACIÓN PASADA")
    print(f"Variables categóricas ({len(categorical_vars)}): {categorical_vars}")
    print(f"Variables numéricas ({len(numeric_vars)}): {numeric_vars[:5]}...")
    print(f"Variables excluidas ({len(forced_excludes)}): {forced_excludes}")
    
    return numeric_vars, categorical_vars, forced_excludes


def encode_categorical_variables(df, categorical_vars):
    """
    Codifica variables categóricas usando LabelEncoder
    """
    print("Codificando variables categóricas...")
    
    label_encoders = {}
    df_encoded = df.copy()
    
    for col in categorical_vars:
        if col in df.columns:
            le = LabelEncoder()
            # Manejar valores nulos
            df_encoded[col] = df_encoded[col].fillna('UNKNOWN')
            df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
            label_encoders[col] = le
            print(f"  {col}: {len(le.classes_)} categorías únicas")
    
    return df_encoded, label_encoders

def clean_numeric_data(df):
    """
    Limpia y convierte datos numéricos para evitar problemas
    """
    print("Limpiando datos numéricos...")
    
    df_clean = df.copy()
    
    # Convertir plan_precios_cuidados de bool a int
    if 'plan_precios_cuidados' in df_clean.columns:
        df_clean['plan_precios_cuidados'] = df_clean['plan_precios_cuidados'].astype(int)
        print("  Convertido plan_precios_cuidados de bool a int")
    
    # Identificar columnas numéricas
    numeric_columns = df_clean.select_dtypes(include=[np.number]).columns
    
    for col in numeric_columns:
        # Reemplazar infinitos por NaN
        df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
        
        # Convertir float64 a float32 para ahorrar memoria y evitar problemas de precisión
        if df_clean[col].dtype == 'float64':
            df_clean[col] = df_clean[col].astype('float32')
    
    print(f"  Limpiadas {len(numeric_columns)} columnas numéricas")
    print("  Convertidos float64 -> float32")
    print("  Reemplazados valores infinitos por NaN")
    
    return df_clean

def standardize_numeric_features(X_train, X_test, numeric_vars):
    """
    Estandariza variables numéricas
    """
    print("Estandarizando variables numéricas...")
    
    scaler = StandardScaler()
    
    # Copiar los datasets
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    # Manejar valores nulos antes de estandarizar
    X_train_scaled = X_train_scaled.fillna(0)
    X_test_scaled = X_test_scaled.fillna(0)
    
    # Identificar columnas numéricas que existen en los datos
    numeric_cols_present = [col for col in numeric_vars if col in X_train.columns]
    
    # Verificar que las columnas sean realmente numéricas y no tengan problemas
    numeric_cols_valid = []
    for col in numeric_cols_present:
        if X_train_scaled[col].dtype in ['int64', 'float64', 'int32', 'float32']:
            # Verificar que no haya infinitos o valores muy grandes
            if not (np.isinf(X_train_scaled[col]).any() or np.isinf(X_test_scaled[col]).any()):
                # Verificar que no haya valores extremadamente grandes
                max_val = max(X_train_scaled[col].abs().max(), X_test_scaled[col].abs().max())
                if max_val < 1e10:  # Límite razonable
                    numeric_cols_valid.append(col)
                else:
                    print(f"  Saltando {col}: valores muy grandes (max: {max_val:.2e})")
            else:
                print(f"  Saltando {col}: contiene valores infinitos")
        else:
            print(f"  Saltando columna no numérica: {col} (tipo: {X_train_scaled[col].dtype})")
    
    if numeric_cols_valid:
        # Estandarizar solo las columnas numéricas válidas
        X_train_scaled[numeric_cols_valid] = scaler.fit_transform(X_train_scaled[numeric_cols_valid])
        X_test_scaled[numeric_cols_valid] = scaler.transform(X_test_scaled[numeric_cols_valid])
        print(f"Estandarizadas {len(numeric_cols_valid)} variables numéricas")
    else:
        print("No se encontraron variables numéricas válidas para estandarizar")
    
    return X_train_scaled, X_test_scaled, scaler

def train_lgbm_model(X_train, y_train, X_val, y_val, categorical_features):
    """
    Entrena modelo LightGBM
    """
    print("Entrenando modelo LightGBM...")
    
    # Parámetros del modelo
    params = {
        #'objective': 'regression',
        #'metric': 'rmse',
        #'boosting_type': 'gbdt',
        #'num_leaves': 50,  # Aumentado
        #'learning_rate': 0.03,  # Reducido
        #'feature_fraction': 0.8,
        #'bagging_fraction': 0.8,
        #'bagging_freq': 5,
        #'min_data_in_leaf': 100,  # NUEVO - evita overfitting
        #'lambda_l1': 0.1,  # NUEVO - regularización
        #'lambda_l2': 0.1,  # NUEVO - regularización
        #'verbose': -1,
        #'random_state': 42,
        #'force_col_wise': True

        'n_jobs': -1, 
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',

        #Mejor corrida

        
        
        
        'num_leaves': 31, # la corrida en en kaggle mejor tiene 31 leaves
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'random_state': 42,
        #'sample_weight': True, 
        #'linear_tree': True,

        #'num_leaves': 63,
        #'learning_rate': 0.03738149205005092,
        #'feature_fraction': 0.5576004633455672,
        #'bagging_fraction': 0.8444560165276432,
        #'bagging_freq': 1,
        #'min_data_in_leaf': 83,  # NUEVO - evita overfitting
        #'lambda_l1': 0.0016447293604905321,
        #'lambda_l2': 3.901069568054151e-06,  # NUEVO - regularización
        #'min_gain_to_split': 0.8073822295272939,  # NUEVO - evita splits innecesarios
        #'max_depth': 11,  # Controla profundidad del árbol
    }
    
    # Crear datasets de LightGBM
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features, reference=train_data)
    
    # Entrenar modelo
    model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'val'],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=100)]
    )
    
    return model


def train_lgbm_model_improved(X_train, y_train, X_val, y_val, categorical_features):
    """
    Entrena modelo LightGBM con hiperparámetros optimizados
    REEMPLAZA tu función train_lgbm_model() existente con esta
    """
    print("Entrenando modelo LightGBM mejorado...")
    
    # Parámetros optimizados para regresión de ventas
    params = {
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        
        # COMPLEJIDAD DEL MODELO
        'num_leaves': 80,               # Aumentado de 31 → captura más patrones
        'max_depth': 7,                 # Controla profundidad del árbol
        'min_data_in_leaf': 30,         # Mínimo de datos por hoja → evita overfitting
        
        # LEARNING RATE
        'learning_rate': 0.025,         # Reducido de 0.05 → convergencia más lenta pero mejor
        
        # SAMPLING Y FEATURES
        'feature_fraction': 0.75,       # 75% de features por árbol → más diversidad
        'bagging_fraction': 0.75,       # 75% de datos por árbol → reduce overfitting  
        'bagging_freq': 3,              # Cada 3 iteraciones → más randomización
        
        # REGULARIZACIÓN
        'lambda_l1': 0.05,              # Regularización L1 → selección de features
        'lambda_l2': 0.05,              # Regularización L2 → suaviza weights
        'min_gain_to_split': 0.05,      # Ganancia mínima para split → evita ruido
        'min_sum_hessian_in_leaf': 20,  # Control adicional de overfitting
        
        # MANEJO DE CATEGORICAS
        'cat_smooth': 10,               # Suavizado para variables categóricas
        'max_cat_threshold': 32,        # Máximo threshold para categóricas
        
        # PERFORMANCE
        'verbose': -1,
        'random_state': 42,
        'force_col_wise': True,
        #'n_jobs': -1,                   # Usar todos los cores
        'deterministic': True           # Para reproducibilidad
    }
    
    # Configuración de entrenamiento
    training_config = {
        'num_boost_round': 2500,        # Más iteraciones para mejor convergencia
        'early_stopping_rounds': 200,   # Más paciencia antes de parar
        'verbose_eval': 100             # Log cada 100 iteraciones
    }
    
    print(f"Configuración del modelo:")
    print(f"  Learning rate: {params['learning_rate']}")
    print(f"  Num leaves: {params['num_leaves']}")
    print(f"  Max depth: {params['max_depth']}")
    print(f"  Feature fraction: {params['feature_fraction']}")
    print(f"  Max iterations: {training_config['num_boost_round']}")
    print(f"  Early stopping: {training_config['early_stopping_rounds']}")
    
    # Crear datasets de LightGBM
    train_data = lgb.Dataset(
        X_train, 
        label=y_train, 
        categorical_feature=categorical_features,
        free_raw_data=False  # Mantener datos para debugging
    )
    
    val_data = lgb.Dataset(
        X_val, 
        label=y_val, 
        categorical_feature=categorical_features, 
        reference=train_data,
        free_raw_data=False
    )
    
    # Entrenar modelo
    model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'val'],
        num_boost_round=training_config['num_boost_round'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=training_config['early_stopping_rounds']), 
            lgb.log_evaluation(period=training_config['verbose_eval'])
        ]
    )
    
    print(f"✅ Entrenamiento completado:")
    print(f"  Mejor iteración: {model.best_iteration}")
    
    # Manejo seguro de best_score (formato puede variar)
    try:
        train_score = model.best_score['train']['rmse']
        val_score = model.best_score['valid_1']['rmse']
        print(f"  Score final train: {train_score:.4f}")
        print(f"  Score final val: {val_score:.4f}")
    except (KeyError, TypeError):
        # Formato alternativo o no disponible
        print(f"  Scores finales disponibles en model.best_score")
        print(f"  Keys disponibles: {list(model.best_score.keys()) if hasattr(model, 'best_score') else 'No disponible'}")
    
    return model

def analyze_model_performance(model, feature_names, X_val, y_val):
    """
    Función opcional para analizar el rendimiento del modelo mejorado
    """
    print("\n=== ANÁLISIS DEL MODELO ===")
    
    # 1. Feature importance
    try:
        importance = model.feature_importance(importance_type='gain')
        feature_imp = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
        print("Top 10 features más importantes:")
        print(feature_imp.head(10)[['feature', 'importance']])
    except Exception as e:
        print(f"Error calculando feature importance: {e}")
        feature_imp = None
    
    # 2. Predicciones de validación
    try:
        y_pred_val = model.predict(X_val)
        
        # 3. Análisis de residuos
        residuals = y_val - y_pred_val
        print(f"\nAnálisis de residuos:")
        print(f"  Media de residuos: {residuals.mean():.4f} (debe estar cerca de 0)")
        print(f"  Std de residuos: {residuals.std():.4f}")
        print(f"  % predicciones negativas: {(y_pred_val < 0).mean()*100:.2f}%")
        
        # 4. Métricas adicionales
        from sklearn.metrics import mean_absolute_error, r2_score
        mae = mean_absolute_error(y_val, y_pred_val)
        r2 = r2_score(y_val, y_pred_val)
        print(f"  MAE validación: {mae:.4f}")
        print(f"  R² validación: {r2:.4f}")
        
    except Exception as e:
        print(f"Error en análisis de predicciones: {e}")
    
    return feature_imp


def evaluate_model(model, X_test, y_test):
    """
    Evalúa el modelo
    """
    print("Evaluando modelo...")
    
    # Predicciones
    y_pred = model.predict(X_test)
    
    # Métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nMétricas del modelo:")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    
    return {'mae': mae, 'rmse': rmse, 'r2': r2, 'predictions': y_pred}

def get_feature_importance(model, feature_names):
    """
    Obtiene importancia de features
    """
    importance = model.feature_importance(importance_type='gain')
    feature_imp = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    print("\nTop 15 features más importantes:")
    print(feature_imp.head(15))
    
    return feature_imp

def verify_model_contamination():
    """
    Verifica si el modelo actual está contaminado
    """
    try:
        import pickle
        with open('model_objects.pkl', 'rb') as f:
            objects = pickle.load(f)
        
        feature_cols = objects['feature_cols']
        
        contaminated = []
        for var in ['product_id', 'customer_id', 'periodo']:
            if var in feature_cols:
                contaminated.append(var)
        
        if contaminated:
            print(f"🚨 MODELO CONTAMINADO: Contiene {contaminated}")
            print("🔄 NECESITAS RE-ENTRENAR con prepare_features_fixed()")
            return True
        else:
            print("✅ Modelo limpio")
            return False
            
    except Exception as e:
        print(f"No se pudo verificar: {e}")
        return True

def main():
    """
    Función principal
    """
    print("=== MODELO LGBM PARA PREDICCIÓN DE TONELADAS ===\n")
    
    # 0. Verificar si el modelo está contaminado
    #if verify_model_contamination():
    #    print("⚠️ El modelo está contaminado. Por favor, re-entrena usando prepare_features_fixed().")
    #    return None, None, None

    # 1. Cargar datos
    df = load_and_prepare_data()
    
    # 2. Crear variable objetivo
    df = create_target_variable_correct(df)
    
    # 2.5. Limpiar datos numéricos (nueva función)
    df = clean_numeric_data(df)
    
    # 3. Filtrar datos para entrenamiento (hasta PERIODO_CORTE)
    train_data = df[df['periodo'] <= PERIODO_CORTE].copy()
    print(f"\nDatos de entrenamiento: {train_data.shape}")
    
    # 4. Remover registros sin target
    train_data = train_data.dropna(subset=['target_tn'])
    print(f"Datos después de remover NaN en target: {train_data.shape}")
    
    # 5. Preparar features
    #numeric_vars, categorical_vars, exclude_vars = prepare_features(train_data)
    numeric_vars, categorical_vars, exclude_vars = prepare_features_fixed(train_data)

    # 6. Codificar variables categóricas
    train_data_encoded, label_encoders = encode_categorical_variables(train_data, categorical_vars)
    
    # 7. Seleccionar features finales
    feature_cols = numeric_vars + categorical_vars
    feature_cols = [col for col in feature_cols if col in train_data_encoded.columns]
    
    print(f"\nFeatures finales: {len(feature_cols)}")
    
    # 8. Preparar X y y
    X = train_data_encoded[feature_cols].copy()
    y = train_data_encoded['target_tn'].copy()
    
    # Manejar valores nulos en X
    X = X.fillna(0)
    
    print(f"Shape final - X: {X.shape}, y: {y.shape}")
    
    # 9. Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=None
    )
    
    # 10. Split train/validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    
    print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    
    # 11. Estandarizar variables numéricas
    X_train_scaled, X_val_scaled, scaler = standardize_numeric_features(X_train, X_val, numeric_vars)
    X_test_scaled, _, _ = standardize_numeric_features(X_test, X_test, numeric_vars)
    
    # 12. Identificar features categóricas para LightGBM
    categorical_indices = [i for i, col in enumerate(feature_cols) if col in categorical_vars]
    
    # 13. Entrenar modelo
    model = train_lgbm_model(
        X_train_scaled, y_train, 
        X_val_scaled, y_val, 
        categorical_indices
    )

    feature_analysis = analyze_model_performance(model, feature_cols, X_val_scaled, y_val)
    
    # 14. Evaluar modelo
    results = evaluate_model(model, X_test_scaled, y_test)
    
    # 15. Feature importance
    feature_importance = get_feature_importance(model, feature_cols)
    
    # 16. Guardar modelo y objetos necesarios
    print("\nGuardando modelo...")
    model.save_model('../output/lgbm/02_lgbm_model.txt')
    
    # Guardar escalador y encoders
    import pickle
    with open('model_objects.pkl', 'wb') as f:
        pickle.dump({
            'scaler': scaler,
            'label_encoders': label_encoders,
            'feature_cols': feature_cols,
            'categorical_vars': categorical_vars,
            'numeric_vars': numeric_vars
        }, f)
    
    print("Modelo guardado como '02_lgbm_model.txt'")
    print("Objetos auxiliares guardados como 'model_objects.pkl'")
    
    return model, results, feature_importance

if __name__ == "__main__":
    model, results, feature_importance = main()

=== MODELO LGBM PARA PREDICCIÓN DE TONELADAS ===



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Datos originales: (17021654, 38)
Periodos únicos: [201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812, 201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912]
Limpiando datos numéricos...
  Convertido plan_precios_cuidados de bool a int
  Limpiadas 36 columnas numéricas
  Convertidos float64 -> float32
  Reemplazados valores infinitos por NaN

Datos de entrenamiento: (17021654, 39)
Datos después de remover NaN en target: (15628837, 39)
Preparando features (versión CORREGIDA)...
✅ VERIFICACIÓN PASADA
Variables categóricas (3): ['cat1', 'cat2', 'brand']
Variables numéricas (30): ['is_max_tn_12m', 'tendencia_6m', 'tn_lag_3m', 'tn_mismo_mes_anio_anterior', 'is_min_tn_6m']...
Variables excluidas (6): ['product_id', 'customer_id', 'periodo', 'target_tn', 'tn', 'registro_sintetico']
Codificando variables categóricas

In [6]:
# =============================================================================
# CHUNK DE PREDICCIÓN CORREGIDO PARA EL NUEVO ENFOQUE
# =============================================================================

import duckdb
import pandas as pd
import numpy as np
import lightgbm as lgb
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

def load_productos_a_predecir():
    """
    Carga la lista de productos a predecir desde DuckDB
    """
    print("Cargando productos a predecir...")
    
    # Conectar a la base de datos DuckDB
    con = duckdb.connect(database='../input/db/labo3.duckdb')
    
    # Cargar productos a predecir
    query = "SELECT * FROM tb_productos_a_predecir"
    productos_df = con.execute(query).df()
    
    con.close()
    
    print(f"Productos a predecir: {len(productos_df)}")
    print(f"Product_ids únicos: {productos_df['product_id'].nunique()}")
    
    return productos_df

def load_latest_data_for_prediction():
    """
    Carga los datos más recientes para hacer predicciones
    """
    print("Cargando datos más recientes para predicción...")
    
    # Conectar a la base de datos DuckDB
    con = duckdb.connect(database='../input/db/labo3.duckdb')
    
    # Cargar datos del periodo 201912 - TODOS los campos como en entrenamiento
    query = """SELECT 
        *
    FROM ventas_features_final 
    WHERE periodo = 201912"""
    
    df = con.execute(query).df()
    con.close()
    
    print(f"Datos del periodo 201912: {df.shape}")
    
    return df

def load_model_and_objects():
    """
    Carga el modelo entrenado y objetos auxiliares
    """
    print("Cargando modelo y objetos auxiliares...")
    
    # Cargar modelo LightGBM
    model = lgb.Booster(model_file='../output/lgbm/02_lgbm_model.txt')
    
    # Cargar objetos auxiliares
    with open('model_objects.pkl', 'rb') as f:
        objects = pickle.load(f)
    
    return model, objects

def clean_numeric_data_prediction(df):
    """
    Aplica la misma limpieza de datos numéricos que en entrenamiento
    """
    print("Limpiando datos numéricos para predicción...")
    
    df_clean = df.copy()
    
    # Convertir plan_precios_cuidados de bool a int
    if 'plan_precios_cuidados' in df_clean.columns:
        df_clean['plan_precios_cuidados'] = df_clean['plan_precios_cuidados'].astype(int)
    
    # Identificar columnas numéricas
    numeric_columns = df_clean.select_dtypes(include=[np.number]).columns
    
    for col in numeric_columns:
        # Reemplazar infinitos por NaN
        df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
        
        # Convertir float64 a float32
        if df_clean[col].dtype == 'float64':
            df_clean[col] = df_clean[col].astype('float32')
    
    print(f"  Limpiadas {len(numeric_columns)} columnas numéricas")
    
    return df_clean

def prepare_prediction_data(df, objects, productos_a_predecir):
    """
    Prepara datos para predicción - SOLO productos solicitados
    """
    print("Preparando datos para predicción...")
    
    # FILTRAR solo productos que necesitamos predecir
    product_ids_to_predict = productos_a_predecir['product_id'].unique()
    df_filtered = df[df['product_id'].isin(product_ids_to_predict)].copy()
    
    print(f"Productos a predecir: {len(product_ids_to_predict)}")
    print(f"Registros filtrados: {df_filtered.shape}")
    
    if df_filtered.empty:
        print("¡ADVERTENCIA! No se encontraron datos para los productos a predecir.")
        return None
    
    # Aplicar limpieza de datos
    df_filtered = clean_numeric_data_prediction(df_filtered)
    
    scaler = objects['scaler']
    label_encoders = objects['label_encoders']
    feature_cols = objects['feature_cols']
    categorical_vars = objects['categorical_vars']
    numeric_vars = objects['numeric_vars']
    
    # Codificar variables categóricas
    for col in categorical_vars:
        if col in df_filtered.columns and col in label_encoders:
            le = label_encoders[col]
            df_filtered[col] = df_filtered[col].fillna('UNKNOWN')
            
            # Manejar categorías no vistas durante el entrenamiento
            unknown_mask = ~df_filtered[col].astype(str).isin(le.classes_)
            df_filtered[col] = df_filtered[col].astype(str)
            df_filtered.loc[unknown_mask, col] = 'UNKNOWN'
            
            # Si 'UNKNOWN' no está en las clases, usar la primera clase
            if 'UNKNOWN' not in le.classes_:
                df_filtered.loc[unknown_mask, col] = le.classes_[0]
            
            df_filtered[col] = le.transform(df_filtered[col])
    
    # Seleccionar features
    missing_features = [col for col in feature_cols if col not in df_filtered.columns]
    if missing_features:
        print(f"¡ADVERTENCIA! Features faltantes: {missing_features}")
        # Crear columnas faltantes con valor 0
        for col in missing_features:
            df_filtered[col] = 0
    
    X_pred = df_filtered[feature_cols].copy()
    
    # Manejar valores nulos
    X_pred = X_pred.fillna(0)
    
    # Estandarizar variables numéricas
    numeric_cols_present = [col for col in numeric_vars if col in X_pred.columns]
    numeric_cols_valid = []
    
    for col in numeric_cols_present:
        if X_pred[col].dtype in ['int64', 'float64', 'int32', 'float32']:
            if not np.isinf(X_pred[col]).any():
                max_val = X_pred[col].abs().max()
                if max_val < 1e10:
                    numeric_cols_valid.append(col)
    
    if numeric_cols_valid:
        X_pred[numeric_cols_valid] = scaler.transform(X_pred[numeric_cols_valid])
    
    # Conservar información para el resultado final
    result_info = df_filtered[['customer_id', 'product_id']].copy()
    
    return X_pred, result_info



def make_predictions_for_products():
    """
    Función principal para hacer predicciones para productos específicos
    """
    print("=== PREDICCIÓN PARA PRODUCTOS ESPECÍFICOS ===\\n")
    
    # 1. Cargar productos a predecir
    productos_a_predecir = load_productos_a_predecir()
    
    # 2. Cargar modelo y objetos
    model, objects = load_model_and_objects()
    
    # 3. Cargar datos más recientes
    df_latest = load_latest_data_for_prediction()
    
    # 4. Preparar datos para predicción (solo productos solicitados)
    prediction_result = prepare_prediction_data(df_latest, objects, productos_a_predecir)
    
    if prediction_result is None:
        print("No se pudieron preparar los datos para predicción.")
        return None
    
    X_pred, result_info = prediction_result
    
    # 5. Hacer predicciones individuales (cliente-producto)
    print("Generando predicciones individuales...")
    predictions = model.predict(X_pred)
    
    # 6. Agregar predicciones al resultado
    result_info['predicted_tn'] = predictions
    
    print(f"Predicciones individuales generadas: {len(predictions)}")
    print(f"Rango de predicciones: {predictions.min():.2f} - {predictions.max():.2f}")
    print(f"Promedio predicción individual: {predictions.mean():.2f}")
    
    # 7. Agrupar por product_id y SUMAR las predicciones
    print("Agrupando predicciones por product_id...")
    final_predictions = result_info.groupby('product_id').agg({
        'predicted_tn': 'sum'
    }).reset_index()
    
    # Renombrar columna para claridad
    final_predictions = final_predictions.rename(columns={'predicted_tn': 'tn'})
    
    # Ordenar por tn descendente
    final_predictions = final_predictions.sort_values('tn', ascending=False)
    
    print(f"Resultados finales:")
    print(f"Productos con predicción: {len(final_predictions)}")
    print(f"Total tn predichas: {final_predictions['tn'].sum():.2f}")
    print(f"Promedio tn por producto: {final_predictions['tn'].mean():.2f}")
    print(f"Máximo tn por producto: {final_predictions['tn'].max():.2f}")
    
    # Mostrar top 10
    print("Top 10 productos con mayor predicción:")
    print(final_predictions.head(10))
    
    # 8. Verificar que tenemos todos los productos solicitados
    productos_solicitados = set(productos_a_predecir['product_id'].unique())
    productos_predichos = set(final_predictions['product_id'].unique())
    productos_faltantes = productos_solicitados - productos_predichos
    
    if productos_faltantes:
        print(f"¡ADVERTENCIA! Productos sin datos en 201912: {len(productos_faltantes)}")
        print(f"Product_ids faltantes: {list(productos_faltantes)[:10]}...")
        
        # Agregar productos faltantes con tn = 0
        for product_id in productos_faltantes:
            final_predictions = pd.concat([
                final_predictions,
                pd.DataFrame({'product_id': [product_id], 'tn': [0.0]})
            ], ignore_index=True)
        
        print(f"Productos faltantes agregados con tn=0")
    
    print(f"Resultado final: {len(final_predictions)} productos")
    




    # 9. Guardar resultados
    output_file = '../output/lgbm/02_lgbm_predicciones.csv'
    final_predictions.to_csv(output_file, index=False)
    print(f"Predicciones guardadas en: {output_file}")
    
    #guardar el contenido del archivo actual en un txt



    return final_predictions
    

# Ejecutar predicciones
if __name__ == "__main__":
    predicciones = make_predictions_for_products()

=== PREDICCIÓN PARA PRODUCTOS ESPECÍFICOS ===\n
Cargando productos a predecir...
Productos a predecir: 780
Product_ids únicos: 780
Cargando modelo y objetos auxiliares...
Cargando datos más recientes para predicción...
Datos del periodo 201912: (553419, 98)
Preparando datos para predicción...
Productos a predecir: 780
Registros filtrados: (465660, 98)
Limpiando datos numéricos para predicción...
  Limpiadas 85 columnas numéricas
Generando predicciones individuales...
Predicciones individuales generadas: 465660
Rango de predicciones: -1.10 - 102.70
Promedio predicción individual: 0.06
Agrupando predicciones por product_id...
Resultados finales:
Productos con predicción: 780
Total tn predichas: 26873.18
Promedio tn por producto: 34.45
Máximo tn por producto: 1292.36
Top 10 productos con mayor predicción:
    product_id           tn
0        20001  1292.356422
2        20003   628.009509
3        20004   586.016164
1        20002   558.464380
4        20005   489.382897
8        20009   4